# BMI INDEX IN IRELAND
## AUTHOR: ANTE DUJIC

## INTRODUCTION

This notebook contains a step by step simulated dataset of a relationship between BMI and Alcohol consumption in Ireland. The data is modeled and synthesised using *numpy.random* package in Python. 

Body mass index (BMI) is a value derived from the mass (weight) and height of a person. The BMI is defined as the body mass divided by the square of the body height, and is expressed in units of kg/m2, resulting from mass in kilograms and height in metres. [1]  It is widely used to diagnose whether individuals are underweight, overweight, or obese. [2] Alcohol consumption is often linked to a high weight. Drinking more than seven times per week was associated with increased risk of weight gain and development of overweight and obesity. [3] Average BMI is increased with an increased level of alcohol consumption in men and women. [4] The goal of this project is to show how to simulate the data based on a real life reasearch. The variables that are simulated are:

1. Name
2. Gender
3. Age
4. Height
5. Weight
6. BMI
7. Alcohol Consumption
8. BMI Classification

Further below it is explained how each of the variables are simulated and what is the relationship between them, if there is any. 

## REFERENCES

- [1] https://en.wikipedia.org/wiki/Body_mass_index
- [2] https://www.hindawi.com/journals/tswj/2012/849018/
- [3] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4338356/
- [4] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6484200/